#Import Libraries

In [ ]:
import librosa
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.models import load_model
from sklearn.feature_selection import SelectKBest, f_classif

##Initiate main settings

In [ ]:
n_mfcc = 40
n_fft = 1024  # setting the FFT size to 1024
hop_length = 10*16 # 25ms*16khz samples has been taken
win_length = 25*16 #25ms*16khz samples has been taken for window length
window = 'hann' #hann window used
n_chroma=12
n_mels=128
n_bands=7 #we are extracting the 7 features out of the spectral contrast
fmin=100
bins_per_ocatve=12

#Upload data and extract features

In [ ]:
# define function to extract features
def extract_features(file_path):
    try:
          # Load audio file and extract features
          y, sr = librosa.load(file_path, sr=16000)
          mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40,n_fft=n_fft,hop_length=hop_length,win_length=win_length,window=window).T,axis=0)
          mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr,n_fft=n_fft, hop_length=hop_length, win_length=win_length, window='hann',n_mels=n_mels).T,axis=0)
          stft = np.abs(librosa.stft(y))
          chroma = np.mean(librosa.feature.chroma_stft(S=stft, y=y, sr=sr).T,axis=0)
          contrast = np.mean(librosa.feature.spectral_contrast(S=stft, y=y, sr=sr,n_fft=n_fft,
                                                        hop_length=hop_length, win_length=win_length,
                                                        n_bands=n_bands, fmin=fmin).T,axis=0)
          tonnetz =np.mean(librosa.feature.tonnetz(y=y, sr=sr).T,axis=0)
          features = np.concatenate((mfcc, chroma, mel, contrast, tonnetz))
          # print(shape(features))
          return features
    except:
        print("Error: Exception occurred in feature extraction")
    return None

# define path to data directory
path = '/content/drive/MyDrive/NLP/donateacry-corpus/donateacry_corpus_cleaned_and_updated_data' #Provide your path here
# initialize empty lists to hold extracted features and labels
features = []
labels = []

# loop through the directories
for label in os.listdir(path):
    # loop through the files in each directory
    print(f"{label} data is loading.....")
    for file_name in os.listdir(os.path.join(path, label)):
        file_path = os.path.join(path, label, file_name)
        # extract features and append to list
        feature = extract_features(file_path)
        if feature is not None:
            features.append(feature)
            labels.append(label)
    print(f"{label} data loaded....")

hungry data is loading.....
hungry data loaded....
tired data is loading.....
tired data loaded....
burping data is loading.....
burping data loaded....
discomfort data is loading.....
discomfort data loaded....
belly_pain data is loading.....
belly_pain data loaded....


In [ ]:
#Encoding results
'''
hungry :4
tired :3
burping :2
discomfert : 1
belly pain: 0
'''

In [ ]:
# convert features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)
selector = SelectKBest(score_func=f_classif, k=10)  # Select top 10 features
le = LabelEncoder()
labels = le.fit_transform(labels)
features_selected = selector.fit_transform(features, labels)
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_selected, labels, test_size=0.25, random_state=42)

#CNN model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# reshape data for 1D CNN
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# create a Sequential model
model = Sequential()

# add Convolutional layers
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

# add Dense layers
model.add(Dense(50, activation='relu'))
model.add(Dense(len(np.unique(y_train)), activation='softmax'))  # number of unique labels

# compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 64  # you can set this value as per your requirement
steps_per_epoch = 50

# train the model
model.fit(X_train, y_train, epochs=100, steps_per_epoch=steps_per_epoch)
# evaluate the model
cnn_acc = model.evaluate(X_test, y_test, verbose=0)[1]
print("CNN Accuracy:", cnn_acc)

Epoch 1/100
50/50 [==============================] - 1s 4ms/step - loss: 1.9271 - accuracy: 0.7536
Epoch 2/100
50/50 [==============================] - 0s 4ms/step - loss: 0.7191 - accuracy: 0.8223
Epoch 3/100
50/50 [==============================] - 0s 4ms/step - loss: 0.6578 - accuracy: 0.8252
Epoch 4/100
50/50 [==============================] - 0s 4ms/step - loss: 0.5946 - accuracy: 0.8453
Epoch 5/100
50/50 [==============================] - 0s 5ms/step - loss: 0.5500 - accuracy: 0.8223
Epoch 6/100
50/50 [==============================] - 0s 4ms/step - loss: 0.5725 - accuracy: 0.8424
Epoch 7/100
50/50 [==============================] - 0s 4ms/step - loss: 0.8794 - accuracy: 0.8166
Epoch 8/100
50/50 [==============================] - 0s 4ms/step - loss: 0.5389 - accuracy: 0.8596
Epoch 9/100
50/50 [==============================] - 0s 4ms/step - loss: 0.5094 - accuracy: 0.8711
Epoch 10/100
50/50 [==============================] - 0s 5ms/step - loss: 0.4095 - accuracy: 0.8711
Epoch 11/

50/50 [==============================] - 0s 835us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
CNN Accuracy: 0.730434775352478


In [ ]:
#validation accuracy and loss
model.fit(X_test, y_test)

4/4 [==============================] - 0s 16ms/step - loss: 0.3185 - accuracy: 0.8609


In [ ]:
model.evaluate(X_test, y_test, verbose=0)[1]

0.8608695864677429

In [ ]:
# save the model
model.save('content/drive/MyDrive/NLP/model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
loaded_model = load_model('/content/drive/MyDrive/NLP/model.h5')

#Testing the model

In [ ]:
# define path to test data directory
test_path = '/content/drive/MyDrive/NLP/final_test'
# initialize an empty list to hold extracted features
test_features = []

# loop through the files in the directory
print("Loading test data.....")
for file_name in os.listdir(test_path):
    file_path = os.path.join(test_path, file_name)
    # extract features and append to list
    feature = extract_features(file_path)
    if feature is not None:
        test_features.append(feature)
print("Test data loaded....")

# convert list to numpy array
test_features = np.array(test_features)
# use the model to make predictions
predictions = model.predict(test_features)

Loading test data.....
Test data loaded....
1/1 [==============================] - 0s 30ms/step


In [ ]:
def predict_label(features):
    # Use the model to predict the label
    prediction = model.predict(np.array([features]))
    # Assuming it's a classification task, the label would be the index of the maximum value
    label = np.argmax(prediction)
    return label

test_data_dir = '/content/drive/MyDrive/NLP/final_test/'

for filename in os.listdir(test_data_dir):
    file_path = os.path.join(test_data_dir, filename)
    if os.path.isfile(file_path):
        # Extract features from the audio file
        features = extract_features(file_path)

        # Predict the label using the extracted features
        label = predict_label(features)

        # Print the audio name and its predicted label
        print(f'Audio Name: {os.path.basename(file_path)}, Predicted Label: {label}')
    else:
        print(f'The file {file_path} does not exist.')

1/1 [==============================] - 0s 31ms/step
Audio Name: Copy of 64acb345-a61e-4ef3-a5a6-cf83c04b83f1-1430058990597-1.7-m-72-dc.wav, Predicted Label: 2
1/1 [==============================] - 0s 31ms/step
Audio Name: Copy of 999bf14b-e417-4b44-b746-9253f81efe38-1430844958178-1.7-m-04-ch.wav, Predicted Label: 2
1/1 [==============================] - 0s 27ms/step
Audio Name: Copy of 837fd072-8704-4196-9ff1-1d2c07886e55-1432429478471-1.7-m-22-dc.wav, Predicted Label: 2
1/1 [==============================] - 0s 28ms/step
Audio Name: Copy of 665BDF6D-2897-49F9-8AD0-978B3B2A0468-1430530218-1.0-m-26-dc.wav, Predicted Label: 2
1/1 [==============================] - 0s 27ms/step
Audio Name: Copy of 79FF400A-97E2-4701-987D-C7C850D5523C-1430089688-1.0-f-48-dc.wav, Predicted Label: 2
1/1 [==============================] - 0s 26ms/step
Audio Name: Copy of 79FF400A-97E2-4701-987D-C7C850D5523C-1430089621-1.0-f-48-dc.wav, Predicted Label: 2
1/1 [==============================] - 0s 25ms/step
Aud